# エージェントのオーケストレーション: ルーチンとハンドオフ

言語モデルを扱う際、多くの場合、優れたパフォーマンスを得るために必要なのは適切なプロンプトと正しいツールだけです。しかし、多くの独特なフローを扱う場合、事態は複雑になることがあります。このクックブックでは、この問題に取り組む一つの方法を説明します。

**ルーチン**と**ハンドオフ**の概念を紹介し、その実装について説明し、シンプルで強力かつ制御可能な方法で複数のエージェントをオーケストレーションする方法を示します。

最後に、これらのアイデアを実装したサンプルリポジトリ[Swarm](https://github.com/openai/swarm)を例とともに提供します。

インポートの設定から始めましょう。

In [32]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
import json


client = OpenAI()

# ルーチン

「ルーチン」という概念は厳密に定義されているわけではなく、むしろ一連のステップの考え方を捉えることを意図しています。具体的には、ルーチンを自然言語での指示のリスト（システムプロンプトで表現します）と、それらを完了するために必要なツールとして定義しましょう。

例を見てみましょう。以下では、カスタマーサービスエージェント向けのルーチンを定義し、ユーザーの問題をトリアージし、修正案を提案するか返金を提供するよう指示しています。また、必要な関数である`execute_refund`と`look_up_item`も定義しています。これをカスタマーサービスルーチン、エージェント、アシスタントなどと呼ぶことができますが、考え方自体は同じです：一連のステップとそれらを実行するためのツールです。

In [7]:
# Customer Service Routine

system_message = (
    "You are a customer support agent for ACME Inc."
    "Always answer in a sentence or less."
    "Follow the following routine with the user:"
    "1. First, ask probing questions and understand the user's problem deeper.\n"
    " - unless the user has already provided a reason.\n"
    "2. Propose a fix (make one up).\n"
    "3. ONLY if not satisfied, offer a refund.\n"
    "4. If accepted, search for the ID and then execute refund."
    ""
)

def look_up_item(search_query):
    """Use to find item ID.
    Search query can be a description or keywords."""

    # return hard-coded item ID - in reality would be a lookup
    return "item_132612938"


def execute_refund(item_id, reason="not provided"):

    print("Summary:", item_id, reason) # lazy summary
    return "success"


ルーチンの主な力は、そのシンプルさと堅牢性にあります。これらの指示には、ステートマシンやコードの分岐のような条件分岐が含まれていることに注目してください。LLMは実際に、小規模から中規模のルーチンに対してこれらのケースを非常に堅牢に処理することができ、さらに「ソフトな」遵守という利点があります。つまり、LLMは行き詰まりに陥ることなく、自然に会話を誘導することができるのです。

## ルーチンの実行

ルーチンを実行するために、以下を行うシンプルなループを実装しましょう：
1. ユーザー入力を取得する
1. ユーザーメッセージを`messages`に追加する
1. モデルを呼び出す
1. モデルの応答を`messages`に追加する

In [ ]:
def run_full_turn(system_message, messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_message}] + messages,
    )
    message = response.choices[0].message
    messages.append(message)

    if message.content: print("Assistant:", message.content)

    return message


messages = []
while True:
    user = input("User: ")
    messages.append({"role": "user", "content": user})

    run_full_turn(system_message, messages)

ご覧のとおり、これは現在関数呼び出しを無視しているので、それを追加しましょう。

モデルは関数を関数スキーマとしてフォーマットする必要があります。便宜上、Python関数を対応する関数スキーマに変換するヘルパー関数を定義できます。

In [13]:
import inspect

def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

例えば：

In [12]:
def sample_function(param_1, param_2, the_third_one: int, some_optional="John Doe"):
    """
    This is my docstring. Call this function when you want.
    """
    print("Hello, world")

schema =  function_to_schema(sample_function)
print(json.dumps(schema, indent=2))

{
  "type": "function",
  "function": {
    "name": "sample_function",
    "description": "This is my docstring. Call this function when you want.",
    "parameters": {
      "type": "object",
      "properties": {
        "param_1": {
          "type": "string"
        },
        "param_2": {
          "type": "string"
        },
        "the_third_one": {
          "type": "integer"
        },
        "some_optional": {
          "type": "string"
        }
      },
      "required": [
        "param_1",
        "param_2",
        "the_third_one"
      ]
    }
  }
}


これで、この関数を使用して、モデルを呼び出す際にツールを渡すことができます。

In [20]:
messages = []

tools = [execute_refund, look_up_item]
tool_schemas = [function_to_schema(tool) for tool in tools]

response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "Look up the black boot."}],
            tools=tool_schemas,
        )
message = response.choices[0].message

message.tool_calls[0].function

Function(arguments='{"search_query":"black boot"}', name='look_up_item')

最後に、モデルがツールを呼び出す際には、対応する関数を実行し、その結果をモデルに返す必要があります。

これは、ツール名をPython関数に対応付ける`tool_map`を作成し、`execute_tool_call`でそれを検索して呼び出すことで実現できます。最終的に、その結果を会話に追加します。

In [21]:
tools_map = {tool.__name__: tool for tool in tools}

def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)

for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            # add result back to conversation 
            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

Assistant: look_up_item({'search_query': 'black boot'})


実際には、モデルがその結果を使用して別の応答を生成できるようにしたいでしょう。その応答にも_また_ツール呼び出しが含まれる可能性があるため、ツール呼び出しがなくなるまでループで実行することができます。

すべてをまとめると、次のようになります：

In [ ]:
tools = [execute_refund, look_up_item]


def run_full_turn(system_message, tools, messages):

    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in tools]
        tools_map = {tool.__name__: tool for tool in tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_message}] + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print assistant response
            print("Assistant:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)


messages = []
while True:
    user = input("User: ")
    messages.append({"role": "user", "content": user})

    new_messages = run_full_turn(system_message, tools, messages)
    messages.extend(new_messages)

ルーチンができたので、さらにステップやツールを追加したいとしましょう。ある程度までは可能ですが、最終的にあまりにも多くの異なるタスクでルーチンを拡張しようとすると、うまく動作しなくなる可能性があります。ここで複数のルーチンという概念を活用できます。ユーザーのリクエストに応じて、それに対応するための適切なステップとツールを持つ正しいルーチンを読み込むことができます。

システム指示とツールを動的に切り替えることは困難に思えるかもしれません。しかし、「ルーチン」を「エージェント」として捉えれば、この**ハンドオフ**の概念により、これらの切り替えを単純に表現できます。つまり、一つのエージェントが会話を別のエージェントに引き継ぐという形で表現できるのです。

# ハンドオフ

**ハンドオフ**を、エージェント（またはルーチン）がアクティブな会話を別のエージェントに引き継ぐことと定義しましょう。これは電話で他の人に転送されるのと似ていますが、この場合、エージェントは以前の会話の完全な知識を持っています！

ハンドオフの動作を確認するために、まずはエージェントの基本クラスを定義することから始めましょう。

In [24]:
class Agent(BaseModel):
    name: str = "Agent"
    model: str = "gpt-4o-mini"
    instructions: str = "You are a helpful Agent"
    tools: list = []

私たちのコードがこれをサポートするようにするために、`run_full_turn`を個別の`system_message`と`tools`の代わりに`Agent`を受け取るように変更できます：

In [25]:
def run_full_turn(agent, messages):

    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in agent.tools]
        tools_map = {tool.__name__: tool for tool in agent.tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.model,
            messages=[{"role": "system", "content": agent.instructions}] + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print assistant response
            print("Assistant:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)

複数のエージェントを簡単に実行できるようになりました：

In [31]:
def execute_refund(item_name):
    return "success"

refund_agent = Agent(
    name="Refund Agent",
    instructions="You are a refund agent. Help the user with refunds.",
    tools=[execute_refund],
)

def place_order(item_name):
    return "success"

sales_assistant = Agent(
    name="Sales Assistant",
    instructions="You are a sales assistant. Sell the user a product.",
    tools=[place_order],
)


messages = []
user_query = "Place an order for a black boot."
print("User:", user_query)
messages.append({"role": "user", "content": user_query})

response = run_full_turn(sales_assistant, messages) # sales assistant
messages.extend(response)


user_query = "Actually, I want a refund." # implicitly refers to the last item
print("User:", user_query)
messages.append({"role": "user", "content": user_query})
response = run_full_turn(refund_agent, messages) # refund agent

User: Place an order for a black boot.
Assistant: place_order({'item_name': 'black boot'})
Assistant: Your order for a black boot has been successfully placed! If you need anything else, feel free to ask!
User: Actually, I want a refund.
Assistant: execute_refund({'item_name': 'black boot'})
Assistant: Your refund for the black boot has been successfully processed. If you need further assistance, just let me know!


素晴らしい！しかし、ここでは手動でハンドオフを行いました。エージェント自身がいつハンドオフを実行するかを決定できるようにしたいと思います。これを実現するシンプルで驚くほど効果的な方法は、`transfer_to_XXX`関数を与えることです。ここで`XXX`は何らかのエージェントを表します。モデルは、ハンドオフを行うのが適切なタイミングでこの関数を呼び出すことを理解できるほど賢いのです！

### ハンドオフ関数

エージェントがハンドオフの_意図_を表現できるようになったので、実際にそれを実行する必要があります。これを行う方法は多数ありますが、特にクリーンな方法が一つあります。

これまでに定義したエージェント関数（`execute_refund`や`place_order`など）は文字列を返し、それがモデルに提供されます。代わりに、転送したいエージェントを示すために`Agent`オブジェクトを返すとどうでしょうか？次のように：

In [ ]:
refund_agent = Agent(
    name="Refund Agent",
    instructions="You are a refund agent. Help the user with refunds.",
    tools=[execute_refund],
)

def transfer_to_refunds():
    return refund_agent

sales_assistant = Agent(
    name="Sales Assistant",
    instructions="You are a sales assistant. Sell the user a product.",
    tools=[place_order],
)

その後、関数レスポンスの戻り値の型をチェックし、それが`Agent`の場合は使用中のエージェントを更新するようにコードを更新できます！さらに、ハンドオフがある場合に備えて、`run_full_turn`は使用中の最新のエージェントを返す必要があります。（これを整理するために`Response`クラスで行うことができます。）

In [ ]:
class Response(BaseModel):
    agent: Optional[Agent]
    messages: list

更新された `run_full_turn` について：

In [ ]:
def run_full_turn(agent, messages):

    current_agent = agent
    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in current_agent.tools]
        tools = {tool.__name__: tool for tool in current_agent.tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.model,
            messages=[{"role": "system", "content": current_agent.instructions}]
            + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print agent response
            print(f"{current_agent.name}:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools, current_agent.name)

            if type(result) is Agent:  # if agent transfer, update current agent
                current_agent = result
                result = (
                    f"Transfered to {current_agent.name}. Adopt persona immediately."
                )

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

    # ==== 3. return last agent used and new messages =====
    return Response(agent=current_agent, messages=messages[num_init_messages:])


def execute_tool_call(tool_call, tools, agent_name):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"{agent_name}:", f"{name}({args})")

    return tools[name](**args)  # call corresponding function with provided arguments

より多くのAgentを使った例を見てみましょう。

In [ ]:
def escalate_to_human(summary):
    """Only call this if explicitly asked to."""
    print("Escalating to human agent...")
    print("\n=== Escalation Report ===")
    print(f"Summary: {summary}")
    print("=========================\n")
    exit()


def transfer_to_sales_agent():
    """User for anything sales or buying related."""
    return sales_agent


def transfer_to_issues_and_repairs():
    """User for issues, repairs, or refunds."""
    return issues_and_repairs_agent


def transfer_back_to_triage():
    """Call this if the user brings up a topic outside of your purview,
    including escalating to human."""
    return triage_agent


triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You are a customer service bot for ACME Inc. "
        "Introduce yourself. Always be very brief. "
        "Gather information to direct the customer to the right department. "
        "But make your questions subtle and natural."
    ),
    tools=[transfer_to_sales_agent, transfer_to_issues_and_repairs, escalate_to_human],
)


def execute_order(product, price: int):
    """Price should be in USD."""
    print("\n\n=== Order Summary ===")
    print(f"Product: {product}")
    print(f"Price: ${price}")
    print("=================\n")
    confirm = input("Confirm order? y/n: ").strip().lower()
    if confirm == "y":
        print("Order execution successful!")
        return "Success"
    else:
        print("Order cancelled!")
        return "User cancelled order."


sales_agent = Agent(
    name="Sales Agent",
    instructions=(
        "You are a sales agent for ACME Inc."
        "Always answer in a sentence or less."
        "Follow the following routine with the user:"
        "1. Ask them about any problems in their life related to catching roadrunners.\n"
        "2. Casually mention one of ACME's crazy made-up products can help.\n"
        " - Don't mention price.\n"
        "3. Once the user is bought in, drop a ridiculous price.\n"
        "4. Only after everything, and if the user says yes, "
        "tell them a crazy caveat and execute their order.\n"
        ""
    ),
    tools=[execute_order, transfer_back_to_triage],
)


def look_up_item(search_query):
    """Use to find item ID.
    Search query can be a description or keywords."""
    item_id = "item_132612938"
    print("Found item:", item_id)
    return item_id


def execute_refund(item_id, reason="not provided"):
    print("\n\n=== Refund Summary ===")
    print(f"Item ID: {item_id}")
    print(f"Reason: {reason}")
    print("=================\n")
    print("Refund execution successful!")
    return "success"


issues_and_repairs_agent = Agent(
    name="Issues and Repairs Agent",
    instructions=(
        "You are a customer support agent for ACME Inc."
        "Always answer in a sentence or less."
        "Follow the following routine with the user:"
        "1. First, ask probing questions and understand the user's problem deeper.\n"
        " - unless the user has already provided a reason.\n"
        "2. Propose a fix (make one up).\n"
        "3. ONLY if not satesfied, offer a refund.\n"
        "4. If accepted, search for the ID and then execute refund."
        ""
    ),
    tools=[execute_refund, look_up_item, transfer_back_to_triage],
)

最後に、これをループで実行できます（これはPythonノートブックでは動作しないため、別のPythonファイルで試すことができます）：

In [ ]:
agent = triage_agent
messages = []

while True:
    user = input("User: ")
    messages.append({"role": "user", "content": user})

    response = run_full_turn(agent, messages)
    agent = response.agent
    messages.extend(response.messages)

# Swarm

概念実証として、これらのアイデアを[Swarm](https://github.com/openai/swarm)というサンプルライブラリにパッケージ化しました。これは例としてのみ提供されており、本番環境で直接使用すべきではありません。ただし、アイデアやコードを自由に参考にして、独自のものを構築してください！